In [ ]:
###TRANFORMATION FOR SURVEY DATA

##TODO:DATE COLUMN AND FORMAT
##TODO: DROP COLUMNS THAT DO NOT START W Q{INT} EXCEPT 'RECORDED DATA'

import pandas as pd
from textblob import TextBlob
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Load the dataset
file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Vickers Onboarding.csv'
data = pd.read_csv(file_path)

def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False


question_texts = data.iloc[0]

# Identify columns to drop
columns_to_drop = []
for col in data.columns:
    if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
        columns_to_drop.append(col)
    else:
        # Check if all entries in the column (except the question row) are numeric
        if pd.to_numeric(data[col][1:], errors='coerce').notna().all():
            columns_to_drop.append(col)
        elif data[col][1:].str.strip().str.lower().isin(['yes', 'no']).all():
            columns_to_drop.append(col)

# Drop identified columns
data.drop(columns=columns_to_drop, inplace=True)

# Drop the first row containing the question texts
data = data[1:]

# Function to check if feedback is valid
def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

# Drop short and NaN feedbacks
for col in data.columns:
    data[col] = data[col].apply(lambda x: x if is_valid_feedback(x) and is_english(x) else None)

# Append question text to each feedback entry in the respective columns
for col in data.columns:
    if col.startswith('Q'):
        question_text = question_texts[col]
        data[col] = data[col].apply(lambda x: f"Vickers: {question_text}: {x}" if pd.notna(x) else x)
        # data[col] = data[col].apply(lambda x: f"{question_text}: {x}" if pd.notna(x) else x)


# Drop columns with all NaN values after feedback processing
data.dropna(axis=1, how='all', inplace=True)

# Convert the dataset from wide to long format without specifying id_vars
long_format_data = data.melt(var_name='Question Code', value_name='Feedback')

# Filter out rows where Feedback is NaN
long_format_data = long_format_data.dropna(subset=['Feedback'])

# Map all feedback to 'Secured Loan' and 'Auto/Car Loan'
long_format_data['Product'] = 'Investments'
long_format_data['Subcategory'] = 'Vickers'
long_format_data['Feedback Category'] = ''  
long_format_data['Sentiment'] = None  # Placeholder for sentiment analysis results
long_format_data['Sentiment Score'] = None
long_format_data['Source'] = 'Product Survey'

# Define a function to determine sentiment using TextBlob
# def determine_sentiment(text):
#     analysis = TextBlob(text)
#     if analysis.sentiment.polarity > 0.1:
#         return 'Positive'
#     elif analysis.sentiment.polarity < -0.1:
#         return 'Negative'
#     else:
#         return 'Neutral'

# # Apply the sentiment analysis function to the 'Feedback' column
# long_format_data['Sentiment'] = long_format_data['Feedback'].apply(determine_sentiment)

# Define the desired column order
desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']

# Reorder columns using reindex
long_format_data = long_format_data.reindex(columns=desired_columns)

# Save the transformed dataset to a CSV file
output_file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/Transformed New/Transformed_Vickers.csv'
long_format_data.to_csv(output_file_path, index=False)

print("Data transformation complete. File saved to:", output_file_path)


In [ ]:

###FORMAT DATE 

file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/Archive/Transformed/categorisation/categorisation1_fake.csv'
df = pd.read_csv(file_path)

def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        # Parse the date assuming it's in the format '13/1/24' or '1/1/24'
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        
        # Format date_obj into '13/01/24'
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:
        return date_str  # Return original value if parsing fails

# Apply the function to format the Date column
df['Date'] = df['Date'].apply(format_date)

# Delete all entries in the "Sentiment" column
df['Sentiment'] = ""

# Define the function to generate random ratings and corresponding sentiments
def generate_sentiment_and_score():
    score = round(np.random.uniform(0.0, 5.0), 1)
    if 0.0 <= score <= 1.0:
        sentiment = 'Frustrated'
    elif 1.1 <= score <= 2.5:
        sentiment = 'Unsatisfied'
    elif 2.6 <= score <= 3.5:
        sentiment = 'Neutral'
    elif 3.6 <= score <= 4.5:
        sentiment = 'Satisfied'
    elif 4.6 <= score <= 5.0:
        sentiment = 'Excited'
    return score, sentiment

# Apply the function to generate random ratings and corresponding sentiments
df['Sentiment Score'], df['Sentiment'] = zip(*df.apply(lambda row: generate_sentiment_and_score(), axis=1))

# display(df)

# Save the modified DataFrame back to a CSV file
output_file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/Archive/Transformed/categorisation/categorisation1_fake_data_w_sentiment_score.csv'
df.to_csv(output_file_path, index=False)
